<a href="https://colab.research.google.com/github/Nusserle/LieDetection/blob/master/LieDetectionVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install fer
!pip install moviepy

     |████████████████████████████████| 819kB 8.7MB/s 
     |████████████████████████████████| 2.3MB 28.1MB/s 


In [0]:
import numpy as np
import pandas as pd
import cv2
from fer import Video
from fer import FER
from moviepy.video.io.VideoFileClip import VideoFileClip
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd drive/My Drive/Shared/BagOfLies/
df = pd.read_csv("Annotations.csv")
df.head()

/content/drive/My Drive/Shared/BagOfLies


,eeg,gaze,image,run,truth,usernum,video
0,./Finalised/User_0/run_0/EEG.csv,./Finalised/User_0/run_0/Gaze.csv,./Finalised/User_0/run_0/image.csv,0,1,0,./Finalised/User_0/run_0/video.mp4
1,./Finalised/User_0/run_1/EEG.csv,./Finalised/User_0/run_1/Gaze.csv,./Finalised/User_0/run_1/image.csv,1,0,0,./Finalised/User_0/run_1/video.mp4
2,./Finalised/User_0/run_2/EEG.csv,./Finalised/User_0/run_2/Gaze.csv,./Finalised/User_0/run_2/image.csv,2,0,0,./Finalised/User_0/run_2/video.mp4
3,./Finalised/User_0/run_3/EEG.csv,./Finalised/User_0/run_3/Gaze.csv,./Finalised/User_0/run_3/image.csv,3,0,0,./Finalised/User_0/run_3/video.mp4
4,./Finalised/User_0/run_4/EEG.csv,./Finalised/User_0/run_4/Gaze.csv,./Finalised/User_0/run_4/image.csv,4,0,0,./Finalised/User_0/run_4/video.mp4


In [5]:
emo_df = pd.DataFrame(columns=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'])
faulty_videos = []

for i in range(len(df['video'])):
  try:
    print(df['video'][i])
    input_video_path = df['video'][i]
    output_video_path = 'video.mp4'

    with VideoFileClip(input_video_path) as video:
        new = video.subclip(3, 4)
        new.write_videofile(output_video_path, audio_codec='aac')

    video_filename = 'video.mp4'
    video = Video(video_filename)

    # Analyze video, displaying the output
    detector = FER()
    raw_data = video.analyze(detector, display=False)
    df_v = video.to_pandas(raw_data)

    emo_dict = {'angry': df_v['angry'].mean(), 'disgust': df_v['disgust'].mean(), 'fear': df_v['fear'].mean(), 'happy': df_v['happy'].mean(), 'sad': df_v['sad'].mean(), 'surprise': df_v['surprise'].mean(), 'neutral': df_v['neutral'].mean()}
    emo_df = emo_df.append(emo_dict, ignore_index=True)
  except:
    faulty_videos.append(df['video'][i])
    continue

emo_df
print(faulty_videos)




./Finalised/User_0/run_0/video.mp4
[MoviePy] >>>> Building video video.mp4
[MoviePy] Writing audio in videoTEMP_MPY_wvf_snd.mp4


100%|██████████| 23/23 [00:00<00:00, 381.54it/s]


[MoviePy] Done.
[MoviePy] Writing video video.mp4


100%|██████████| 31/31 [00:00<00:00, 61.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video.mp4 




...

[MoviePy] Done.
[MoviePy] Writing video video.mp4



100%|██████████| 15/15 [00:00<00:00, 120.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video.mp4 



18-01-2020:11:21:00,170 INFO     [classes.py:207] 14.33 fps, 15 frames, 1.05 seconds
18-01-2020:11:21:00,178 INFO     [classes.py:323] Deleted pre-existing output/video_output.mp4
18-01-2020:11:21:27,696 INFO     [classes.py:307] Completed analysis: saved to output/video_output.mp4


['./Finalised/User_12/run_9/video.mp4', './Finalised/User_17/run_4/video.mp4', './Finalised/User_21/run_0/video.mp4', './Finalised/User_22/run_0/video.mp4', './Finalised/User_22/run_1/video.mp4', './Finalised/User_22/run_2/video.mp4', './Finalised/User_22/run_4/video.mp4']


In [67]:
# let's look at the data
emo_df

,angry,disgust,fear,happy,sad,surprise,neutral
0,0.014516,0.0,0.000000,0.020968,0.040323,0.000323,0.920000
1,0.015484,0.0,0.000000,0.068387,0.015161,0.002581,0.895484
2,0.026000,0.0,0.002000,0.039000,0.021333,0.003000,0.904333
3,0.038000,0.0,0.010667,0.083667,0.045667,0.024667,0.799333
4,0.013000,0.0,0.003333,0.030000,0.029000,0.004000,0.919000
...,...,...,...,...,...,...,...
313,0.056667,0.0,0.036667,0.222667,0.052000,0.099333,0.536000
314,0.022000,0.0,0.006000,0.718000,0.030000,0.018667,0.206000
315,0.057333,0.0,0.041333,0.177333,0.207333,0.020000,0.496000
316,0.053333,0.0,0.028000,0.221333,0.146000,0.026667,0.523333


In [64]:
# add most common values to list to fill in the video slots in df that could not be processed correctly with
most_common_list = [df_v['angry'].value_counts().head(1).keys()[0], 
               df_v['disgust'].value_counts().head(1).keys()[0], 
               df_v['fear'].value_counts().head(1).keys()[0], 
               df_v['happy'].value_counts().head(1).keys()[0], 
               df_v['sad'].value_counts().head(1).keys()[0], 
               df_v['surprise'].value_counts().head(1).keys()[0], 
               df_v['neutral'].value_counts().head(1).keys()[0]
               ]

print(most_common_list)

values = emo_df[['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']].to_numpy()

print(len(values))

for i in range(len(df['video'])):
  if df['video'][i] in faulty_videos:
    values = np.insert(values, i, most_common_list, axis=0)

print(len(values))
print(len(df['video']))

df_final = pd.DataFrame(data=values, columns=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'])
df_final['truth'] = df['truth']

df_final


[0.029999999329447746, 0.0, 0.009999999776482582, 0.09000000357627869, 0.09000000357627869, 0.009999999776482582, 0.7599999904632568]
318
325
325


,angry,disgust,fear,happy,sad,surprise,neutral,truth
0,0.014516,0.0,0.000000,0.020968,0.040323,0.000323,0.920000,1
1,0.015484,0.0,0.000000,0.068387,0.015161,0.002581,0.895484,0
2,0.026000,0.0,0.002000,0.039000,0.021333,0.003000,0.904333,0
3,0.038000,0.0,0.010667,0.083667,0.045667,0.024667,0.799333,0
4,0.013000,0.0,0.003333,0.030000,0.029000,0.004000,0.919000,0
...,...,...,...,...,...,...,...,...
320,0.056667,0.0,0.036667,0.222667,0.052000,0.099333,0.536000,0
321,0.022000,0.0,0.006000,0.718000,0.030000,0.018667,0.206000,0
322,0.057333,0.0,0.041333,0.177333,0.207333,0.020000,0.496000,0
323,0.053333,0.0,0.028000,0.221333,0.146000,0.026667,0.523333,0


In [0]:
# create csv file with the data so we could just use the file in the future with the data instead of extracting it again all over again
df_final.to_csv('emotion_data.csv', index=False)

In [66]:
# read the data in from the file
data = pd.read_csv('emotion_data.csv')

data

,angry,disgust,fear,happy,sad,surprise,neutral,truth
0,0.014516,0.0,0.000000,0.020968,0.040323,0.000323,0.920000,1
1,0.015484,0.0,0.000000,0.068387,0.015161,0.002581,0.895484,0
2,0.026000,0.0,0.002000,0.039000,0.021333,0.003000,0.904333,0
3,0.038000,0.0,0.010667,0.083667,0.045667,0.024667,0.799333,0
4,0.013000,0.0,0.003333,0.030000,0.029000,0.004000,0.919000,0
...,...,...,...,...,...,...,...,...
320,0.056667,0.0,0.036667,0.222667,0.052000,0.099333,0.536000,0
321,0.022000,0.0,0.006000,0.718000,0.030000,0.018667,0.206000,0
322,0.057333,0.0,0.041333,0.177333,0.207333,0.020000,0.496000,0
323,0.053333,0.0,0.028000,0.221333,0.146000,0.026667,0.523333,0


In [72]:
X = data[['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']]
y = data['truth']

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train)

        angry   disgust      fear     happy       sad  surprise   neutral
1    0.015484  0.000000  0.000000  0.068387  0.015161  0.002581  0.895484
250  0.317500  0.000000  0.101667  0.054167  0.330000  0.075833  0.118333
8    0.015000  0.000000  0.001000  0.041333  0.016333  0.000000  0.923000
212  0.052667  0.000000  0.033667  0.000000  0.035000  0.017333  0.861667
291  0.095581  0.000000  0.065116  0.045349  0.143721  0.001163  0.645581
..        ...       ...       ...       ...       ...       ...       ...
251  0.096333  0.000667  0.074000  0.003000  0.161667  0.033333  0.629667
192  0.064667  0.008667  0.049333  0.255333  0.191333  0.010000  0.419667
117  0.008387  0.000000  0.002258  0.478710  0.006774  0.026129  0.474839
47   0.393667  0.000000  0.190333  0.107000  0.094333  0.104333  0.110000
172  0.080333  0.001000  0.040333  0.009000  0.081000  0.011333  0.774000

[260 rows x 7 columns]


In [74]:
# create simple model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(7,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
	keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=50, batch_size=1)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Train on 260 samples
Epoch 1/50
260/260 [==============================] - 21s 82ms/sample - loss: 0.6944 - acc: 0.4923
Epoch 2/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6916 - acc: 0.5154
Epoch 3/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6880 - acc: 0.5538
Epoch 4/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6849 - acc: 0.5577
Epoch 5/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6825 - acc: 0.5385
Epoch 6/50
260/260 [==============================] - 12s 46ms/sample - loss: 0.6811 - acc: 0.4962
Epoch 7/50
260/260 [==============================] - 12s 46ms/sample - loss: 0.6788 - acc: 0.5269
Epoch 8/50
260/260 [==============================] - 12s 46ms/sample - loss: 0.6789 - acc: 0.5231
Epoch 9/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6775 - acc: 0.5538
Epoch 10/50
260/260 [==============================] - 12s 45ms/sample - loss: 0.6772 - 